<a href="https://colab.research.google.com/github/amien1410/amien-scrapers/blob/main/Indeed_Succeed_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

In [ ]:
# @title
api_headers = {
    "Host": "apis.indeed.com",
    "content-type": "application/json",
    "indeed-api-key": "161092c2017b5bbab13edb12461a62d5a833871e7cad6d9d475304573de67ac8",
    "accept": "application/json",
    "indeed-locale": "en-US",
    "accept-language": "en-US,en;q=0.9",
    "user-agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_6_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 Indeed App 193.1",
    "indeed-app-info": "appv=193.1; appid=com.indeed.jobsearch; osv=16.6.1; os=ios; dtype=phone",
}
job_search_query = """
    query GetJobData {{
      jobSearch(
        {what}
        {location}
        includeSponsoredResults: NONE
        limit: 100
        sort: DATE
        {cursor}
        {filters}
      ) {{
        pageInfo {{
          nextCursor
        }}
        results {{
          trackingKey
          job {{
            key
            title
            datePublished
            dateOnIndeed
            description {{
              html
            }}
            location {{
              countryName
              countryCode
              admin1Code
              city
              postalCode
              streetAddress
              formatted {{
                short
                long
              }}
            }}
            compensation {{
              baseSalary {{
                unitOfWork
                range {{
                  ... on Range {{
                    min
                    max
                  }}
                }}
              }}
              currencyCode
            }}
            attributes {{
              key
              label
            }}
            employer {{
              relativeCompanyPageUrl
              name
              dossier {{
                  employerDetails {{
                    addresses
                    industry
                    employeesLocalizedLabel
                    revenueLocalizedLabel
                    briefDescription
                    ceoName
                    ceoPhotoUrl
                  }}
                  images {{
                        headerImageUrl
                        squareLogoUrl
                  }}
                  links {{
                    corporateWebsite
                }}
              }}
            }}
            recruit {{
              viewJobUrl
              detailedSalary
              workSchedule
            }}
          }}
        }}
      }}
    }}
    """

In [ ]:
api_url = "https://apis.indeed.com/graphql"
search_term = "data science"
location = "Indonesia"
distance = 100000
hours_old = 72  # (only LinkedIn/Indeed is hour specific, others round up to days old)
cursor = None

def extract_job_details(job_data):
    # Helper function to convert timestamp to readable date
    def convert_timestamp_to_date(timestamp):
        return datetime.datetime.fromtimestamp(timestamp / 1000).strftime('%Y-%m-%d') if timestamp else None

    # Extracting details with additional checks
    job = job_data.get('job', {})

    job_id = job.get('key', 'N/A')
    job_title = job.get('title', 'N/A')
    date_published_timestamp = job.get('datePublished')
    date_published = convert_timestamp_to_date(date_published_timestamp)

    # Handling nested employer field
    employer = job.get('employer')
    company = employer.get('name', 'N/A') if employer else 'N/A'

    # Handling nested compensation field
    compensation = job.get('compensation', {})
    salary = compensation.get('baseSalary', 'N/A')

    # Handling nested location field
    location_info = job.get('location', {})
    street_address = location_info.get('streetAddress', 'N/A')
    city = location_info.get('city', 'N/A')

    # Handling job type
    attributes = job.get('attributes', [])
    job_type = next((attr.get('label') for attr in attributes if attr.get('key') == 'CF3CP'), 'N/A')

    # Handling job URL
    recruit_info = job.get('recruit', {})
    job_url = recruit_info.get('viewJobUrl', 'N/A')

    # Returning extracted details as a dictionary
    return {
        'ID': job_id,
        'Job Title': job_title,
        'Date Published': date_published,
        'Company': company,
        'Salary': salary,
        'Street Address': street_address,
        'City': city,
        'Job Type': job_type,
        'Job URL': job_url
    }

extracted_jobs = []

while True:
    query = job_search_query.format(
        what=(f'what: "{search_term}"' if search_term else ""),
        location=(f'location: {{where: "{location}", radius: {distance}, radiusUnit: MILES}}' if location else ""),
        dateOnIndeed=hours_old,
        cursor=f'cursor: "{cursor}"' if cursor else "",
        filters=""
    )
    payload = {
        "query": query,
    }
    api_headers["indeed-co"] = "ID"
    res = requests.post(
        api_url,
        headers=api_headers,
        json=payload,
        timeout=10,
    )

    jobs = res.json()["data"]["jobSearch"]["results"]
    next_cursor = res.json()['data']['jobSearch']['pageInfo']['nextCursor']

    for job in jobs:
        job_details = extract_job_details(job)
        extracted_jobs.append(job_details)

    if not next_cursor:
        break

    cursor = next_cursor

# Convert extracted job details to DataFrame
df = pd.DataFrame(extracted_jobs)

df

,ID,Job Title,Date Published,Company,Salary,Street Address,City,Job Type,Job URL
0,e42885f4b5375a5f,Sr. Data Analyst,2024-07-15,Flip,None,None,Jakarta,Full-time,https://career.flip.id/jobs/zwxBCpJAUUsp/sr-da...
1,64bbd4e13507a0f7,Dynamics 365 F&O Technical Developer,2024-07-15,IMPACK PRATAMA INDUSTRI,None,None,Jakarta,N/A,https://www.impack-pratama.com/career/#
2,dc34bf00724ab2da,Enterprise Security Associate,2024-07-14,PT. Alto Network,None,None,Jakarta,Full-time,https://alto.recruiterpal.com/career/jobs/gl6q2
3,3f4d9221034df020,Cyber Defense Incident Responder,2024-07-14,PT. Alto Network,None,None,Jakarta,N/A,https://alto.recruiterpal.com/career/jobs/59bwl
4,92b499ae27d544f5,Quality Assurance Engineer Associate,2024-07-14,PT. Alto Network,None,None,Jakarta,Full-time,https://alto.recruiterpal.com/career/jobs/ae7or
...,...,...,...,...,...,...,...,...,...
823,4000c729c3d36a74,Software Developer / Engineer (Internship),2018-11-19,Pinnacle Investment,None,None,Jakarta,N/A,https://www.pinnacleinvestment.co.id/current-o...
824,e75543a0a34bbac7,SYSTEM ANALYST / SENIOR DATA SCIENTIST,2018-11-19,PT. Kuryotech Indonesia,None,None,Surabaya,N/A,https://kuryotech.com/career.html#9
825,c58507fbd0002b7b,Quality Assurance,2018-10-16,Weekend Inc,None,None,Jakarta,N/A,https://www.weekendinc.com/career/quality_assu...
826,d87ef6eba6ed2b71,Sr. Digital Strategist/Analyst,2018-10-16,Weekend Inc,None,None,Jakarta,N/A,https://www.weekendinc.com/career/digital_analyst
